In [25]:
import time
import pandas as pd
import numpy as np
import requests
import os
from dotenv import load_dotenv
import re

In [3]:
load_dotenv()

True

In [4]:
# busan_restrua_2019_dir = "/mnt/t7/dnn/llm_practicing/05_RAG_practicing/datas/BC_GUGUN_PRGN_RSTRT_STTUS_201912.csv"
busan_restrua_2020_dir = "/mnt/t7/dnn/llm_practicing/05_RAG_practicing/datas/BC_GUGUN_PRGN_RSTRT_STTUS_20200720.csv"
busan_restrua_2021_dir = "/mnt/t7/dnn/llm_practicing/05_RAG_practicing/datas/BC_GUGUN_PRGN_RSTRT_STTUS_202106.csv"
busan_restrua_2022_dir = "/mnt/t7/dnn/llm_practicing/05_RAG_practicing/datas/BC_GUGUN_PRGN_RSTRT_STTUS_2022.csv"
busan_restrua_2023_dir = "/mnt/t7/dnn/llm_practicing/05_RAG_practicing/datas/BC_GUGUN_PRGN_RSTRT_STTUS_2023.csv"

# restaurants_19 = pd.read_csv(busan_restrua_2019_dir)
restaurants_20 = pd.read_csv(busan_restrua_2020_dir)
restaurants_21 = pd.read_csv(busan_restrua_2021_dir)
restaurants_22 = pd.read_csv(busan_restrua_2022_dir)
restaurants_23 = pd.read_csv(busan_restrua_2023_dir)

In [27]:
restaurants_23.head()

,SYS_NO,INDUTY_NM,BZCND_NM,RSTRNT_NM,LOCPLC_ALL_ADDR,LOCPLC_LNM_ADDR,MENU_NM,TEL_NO,APPN_DE,RSNMT_DE,GUGUN_NM,DATA_BASE_DE,GID_LA,GID_LO
0,1,일반음식점,한식,동백삼계탕,"부산광역시 해운대구 마린시티3로 23 (우동, 벽산e오렌지상가 320호 - 325호 )",부산광역시 해운대구 우동 1435,삼계탕,051-900-9933,2020-11-26,2020-11-26,부산광역시 해운대구,2023-12-31,35.156644,129.146927
1,2,일반음식점,한식,수비가든,부산광역시 해운대구 해운대로469번길 142 (우동),부산광역시 해운대구 우동 1072-38,오리구이,051-742-8998,2020-11-26,2020-11-26,부산광역시 해운대구,2023-12-31,35.168695,129.141356
2,3,일반음식점,한식,신토불이보쌈,"부산광역시 해운대구 구남로29번길 35 (중동, 2층)",부산광역시 해운대구 중동 1380-4,보쌈,051-731-1441,2020-11-26,2020-11-26,부산광역시 해운대구,2023-12-31,35.162818,129.162162
3,4,일반음식점,한식,해운마루,"부산광역시 해운대구 달맞이길62번길 65 (중동, 지상1층)",부산광역시 해운대구 중동 996-1,생선회,051-743-2772,2020-11-26,2020-11-26,부산광역시 해운대구,2023-12-31,35.157001,129.173625
4,5,일반음식점,회집,거북선횟집,부산광역시 해운대구 달맞이길62번길 69 (중동),부산광역시 해운대구 중동 996-7,생선회,051-741-8851,2020-11-26,2020-11-26,부산광역시 해운대구,2023-12-31,35.156827,129.173963


In [5]:
df_list = [restaurants_23, restaurants_22, restaurants_21, restaurants_20]
busan_restrau_df = pd.concat(df_list)

In [7]:
busan_restrau_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2058 entries, 0 to 624
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SYS_NO           2058 non-null   int64  
 1   INDUTY_NM        2058 non-null   object 
 2   BZCND_NM         2058 non-null   object 
 3   RSTRNT_NM        2056 non-null   object 
 4   LOCPLC_ALL_ADDR  2056 non-null   object 
 5   LOCPLC_LNM_ADDR  1854 non-null   object 
 6   MENU_NM          1850 non-null   object 
 7   TEL_NO           1816 non-null   object 
 8   APPN_DE          1501 non-null   object 
 9   RSNMT_DE         1501 non-null   object 
 10  GUGUN_NM         1816 non-null   object 
 11  DATA_BASE_DE     1501 non-null   object 
 12  GID_LA           1816 non-null   float64
 13  GID_LO           1816 non-null   float64
dtypes: float64(2), int64(1), object(11)
memory usage: 241.2+ KB


In [8]:
busan_restrau_df = busan_restrau_df.drop_duplicates(subset='RSTRNT_NM', keep='first').reset_index(drop=True)

In [13]:
busan_restrau_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SYS_NO           627 non-null    int64  
 1   INDUTY_NM        627 non-null    object 
 2   BZCND_NM         627 non-null    object 
 3   RSTRNT_NM        626 non-null    object 
 4   LOCPLC_ALL_ADDR  626 non-null    object 
 5   LOCPLC_LNM_ADDR  569 non-null    object 
 6   MENU_NM          566 non-null    object 
 7   TEL_NO           555 non-null    object 
 8   APPN_DE          451 non-null    object 
 9   RSNMT_DE         451 non-null    object 
 10  GUGUN_NM         555 non-null    object 
 11  DATA_BASE_DE     451 non-null    object 
 12  GID_LA           555 non-null    float64
 13  GID_LO           555 non-null    float64
dtypes: float64(2), int64(1), object(11)
memory usage: 68.7+ KB


In [22]:
none_id_dir = "/mnt/t7/dnn/llm_practicing/05_RAG_practicing/datas/busan_restrau_20to24_none_id_data.csv"

none_id_df = pd.read_csv(none_id_dir)

In [24]:
none_id_df.head()

,MENU_TYPE,RESTAURANT_NAME_KOREAN,ADDRESS_KOREAN,MENU_NAME,NATIONAL_PHONE_NUMBER,DISTRICT_NAME,LATITUDE,LONGITUDE
0,중국식,팔선각,부산광역시 연제구 법원로32번길 10 (거제동),중식,051-507-9555,연제구,35.191,129.075
1,탕류(보신용),토곡정,부산광역시 연제구 과정로 112-1 (연산동),삼계탕,051-755-5666,연제구,35.181,129.108
2,한식,토곡식육식당,부산광역시 연제구 토곡로 7 (연산동),소고기 모듬구이,051-751-9280,연제구,35.183,129.106
3,한식,청담동 대구뽈찜 아구찜,"부산광역시 연제구 월드컵대로120번길 12 (연산동,1층)",아구찜,051-866-8008,연제구,35.185,129.083
4,회집,원조메기탕,부산광역시 연제구 거제시장로 15 (거제동),메기탕,051-852-8085,연제구,35.181,129.072


In [31]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [37]:
def get_place_id(restaurant_name,location):
    search_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    search_params = {
        'input': restaurant_name,
        'inputtype': 'textquery',
        'fields': 'place_id',
        'key': GOOGLE_API_KEY,
        'locationbias': f'circle:2000@{location}'
    }
    response = requests.get(search_url, params=search_params)
    results = response.json().get('candidates', [])
    print(response.json())
    if results:
        return results[0]['place_id']
    else:
        return None
    
# Function to fetch detailed information using the place ID
def get_place_details_new(place_id):
    details_url = f"https://places.googleapis.com/v1/places/{place_id}"
    details_params = {
        'fields': 'displayName,types,formattedAddress,priceLevel,rating,userRatingCount,regularOpeningHours,accessibilityOptions,goodForChildren,goodForGroups,menuForChildren,parkingOptions,paymentOptions,outdoorSeating,reservable,restroom,servesBeer,servesBreakfast,servesBrunch,servesDinner,servesLunch,servesVegetarianFood,takeout,reviews',
        'key': GOOGLE_API_KEY
    }
    response = requests.get(details_url, params=details_params)
    return response.json()

In [36]:
for i, row in none_id_df.iterrows():
    # time.sleep(0.1)
    restaurant_name = row["RESTAURANT_NAME_KOREAN"]
    location = f"{row['LATITUDE'],row['LONGITUDE']}" 
    print(f"식당이름: {restaurant_name}, 좌표: {location}")
    place_id = get_place_id(restaurant_name, location)
    if i > 20:
        break

식당이름: 팔선각, 좌표: (35.191, 129.075)
{'candidates': [{'place_id': 'ChIJNVgZxRqjfDURWvKbkSRzZa4'}], 'status': 'OK'}
식당이름: 토곡정, 좌표: (35.181, 129.108)
{'candidates': [{'place_id': 'ChIJb0pWNR-TaDURHAGBWI1Z5j4'}, {'place_id': 'ChIJ7SwAsxmTaDURbl60WHapGmE'}, {'place_id': 'ChIJh5OAJSOTaDURIAdNicst2L8'}, {'place_id': 'ChIJfQZuVB-TaDUR5PanfBGrTo0'}, {'place_id': 'ChIJpcP-vSKTaDURfcVvioVgHDs'}, {'place_id': 'ChIJG1bb3T-TaDURpOJXD8NsVkg'}], 'status': 'OK'}
식당이름: 토곡식육식당, 좌표: (35.183, 129.106)
{'candidates': [], 'status': 'ZERO_RESULTS'}
식당이름: 청담동 대구뽈찜 아구찜, 좌표: (35.185, 129.083)
{'candidates': [], 'status': 'ZERO_RESULTS'}
식당이름: 원조메기탕, 좌표: (35.181, 129.072)
{'candidates': [], 'status': 'ZERO_RESULTS'}
식당이름: 요풍각, 좌표: (35.183, 129.075)
{'candidates': [], 'status': 'ZERO_RESULTS'}
식당이름: 양말례 남원추어탕, 좌표: (35.182, 129.075)
{'candidates': [], 'status': 'ZERO_RESULTS'}
식당이름: 산청한방삼계탕, 좌표: (35.19, 129.074)
{'candidates': [], 'status': 'ZERO_RESULTS'}
식당이름: 동원숯불갈비, 좌표: (35.189, 129.097)
{'candidates': [], 'status

In [35]:
for i, row in none_id_df.iterrows():
    print(row.to_frame().T)

  MENU_TYPE RESTAURANT_NAME_KOREAN              ADDRESS_KOREAN MENU_NAME  \
0       중국식                    팔선각  부산광역시 연제구 법원로32번길 10 (거제동)        중식   

  NATIONAL_PHONE_NUMBER DISTRICT_NAME LATITUDE LONGITUDE  
0          051-507-9555           연제구   35.191   129.075  
  MENU_TYPE RESTAURANT_NAME_KOREAN             ADDRESS_KOREAN MENU_NAME  \
1   탕류(보신용)                    토곡정  부산광역시 연제구 과정로 112-1 (연산동)       삼계탕   

  NATIONAL_PHONE_NUMBER DISTRICT_NAME LATITUDE LONGITUDE  
1          051-755-5666           연제구   35.181   129.108  
  MENU_TYPE RESTAURANT_NAME_KOREAN         ADDRESS_KOREAN MENU_NAME  \
2        한식                 토곡식육식당  부산광역시 연제구 토곡로 7 (연산동)  소고기 모듬구이   

  NATIONAL_PHONE_NUMBER DISTRICT_NAME LATITUDE LONGITUDE  
2          051-751-9280           연제구   35.183   129.106  
  MENU_TYPE RESTAURANT_NAME_KOREAN                    ADDRESS_KOREAN  \
3        한식           청담동 대구뽈찜 아구찜  부산광역시 연제구 월드컵대로120번길 12 (연산동,1층)   

  MENU_NAME NATIONAL_PHONE_NUMBER DISTRICT_NAME LATITU

In [38]:
get_place_details_new('ChIJNVgZxRqjfDURWvKbkSRzZa4')

{'types': ['chinese_restaurant',
  'restaurant',
  'food',
  'point_of_interest',
  'establishment'],
 'formattedAddress': '249 Dongho-ro, 장충동 Jung District, Seoul, South Korea',
 'rating': 4.4,
 'regularOpeningHours': {'openNow': True,
  'periods': [{'open': {'day': 0, 'hour': 0, 'minute': 0},
    'close': {'day': 0, 'hour': 14, 'minute': 30}},
   {'open': {'day': 0, 'hour': 17, 'minute': 30},
    'close': {'day': 0, 'hour': 21, 'minute': 30}},
   {'open': {'day': 1, 'hour': 0, 'minute': 0},
    'close': {'day': 1, 'hour': 14, 'minute': 30}},
   {'open': {'day': 1, 'hour': 17, 'minute': 30},
    'close': {'day': 1, 'hour': 21, 'minute': 30}},
   {'open': {'day': 2, 'hour': 0, 'minute': 0},
    'close': {'day': 2, 'hour': 14, 'minute': 30}},
   {'open': {'day': 2, 'hour': 17, 'minute': 30},
    'close': {'day': 2, 'hour': 21, 'minute': 30}},
   {'open': {'day': 3, 'hour': 0, 'minute': 0},
    'close': {'day': 3, 'hour': 14, 'minute': 30}},
   {'open': {'day': 3, 'hour': 17, 'minute': 3

In [42]:
def get_place_id(restaurant_name,location):
    search_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    search_params = {
        'input': restaurant_name,
        'inputtype': 'textquery',
        'fields': 'place_id',
        'key': GOOGLE_API_KEY,
        'locationbias': f'circle:2000@{location}'
    }
    response = requests.get(search_url, params=search_params)
    results = response.json().get('candidates', [])
    print(response.json())
    if results:
        return results[0]['place_id']
    else:
        return None

restaurant_name = "팔선각"
location = "35.191, 129.075" 

# Step 1: Get Place ID for the restaurant
place_id = get_place_id(restaurant_name, location)

{'candidates': [{'place_id': 'ChIJ6d5_N8CSaDURdI0rGZNBbQg'}], 'status': 'OK'}


In [66]:
# def get_place_id(api_key, name, lat, lng):
#     url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#     params = {
#         "key": api_key,
#         "location": f"{lat},{lng}",
#         "radius": 100,  # 검색 반경(미터 단위)
#         "keyword": name,  # 레스토랑 이름
#         "type": "restaurant"
#     }
    
#     # Google Places API 요청
#     response = requests.get(url, params=params)
#     if response.status_code == 200:
#         data = response.json()
#         # 검색 결과가 있을 경우 place_id 반환
#         if "results" in data and len(data["results"]) > 0:
#             return data["results"][0]["place_id"]
#         else:
#             return "검색 결과 없음"
#     else:
#         return f"에러: {response.status_code}"
    
def clean_ktext(text):
    cleaned_text = re.sub(r"[^\w가-힣]", "", text)
    return cleaned_text


def get_place_id(api_key, name, lat, lng):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "key": api_key,
        "location": f"{lat},{lng}",
        "radius": 100,  # 검색 반경 (미터 단위)
        "keyword": name,  # 레스토랑 이름
        "type": "restaurant"
    }
    
    # Google Places API 요청
    response = requests.get(url, params=params)
    print(response.json())
    if response.status_code == 200:
        data = response.json()
        
        # 검색 결과가 있을 경우 이름 일치 여부 확인
        if "results" in data and len(data["results"]) > 0:
            for result in data["results"]:
                # 결과의 이름과 입력한 이름이 정확히 일치하는지 확인
                if clean_ktext(result["name"]) == clean_ktext(name):
                    return result["place_id"]
        
        return "검색 결과 없음"
    else:
        return f"에러: {response.status_code}"

# 사용 예시
api_key = GOOGLE_API_KEY  # API 키 입력

# 돌집, 좌표: (35.191, 129.075)
name = "돌집"
lat = 35.191  # 예시 위도
lng = 129.075  # 예시 경도

place_id = get_place_id(api_key, name, lat, lng)
print("Place ID:", place_id)


{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 35.191442, 'lng': 129.0754468}, 'viewport': {'northeast': {'lat': 35.19279182989272, 'lng': 129.0767966298927}, 'southwest': {'lat': 35.19009217010727, 'lng': 129.0740969701072}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'icon_background_color': '#FF9E67', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet', 'name': '돌집', 'place_id': 'ChIJW6Rmul6TaDURleb5ghiERB4', 'plus_code': {'compound_code': '53RG+H5 Busan', 'global_code': '8Q7F53RG+H5'}, 'rating': 4.1, 'reference': 'ChIJW6Rmul6TaDURleb5ghiERB4', 'scope': 'GOOGLE', 'types': ['restaurant', 'food', 'point_of_interest', 'establishment'], 'user_ratings_total': 14, 'vicinity': 'Busan'}], 'status': 'OK'}
Place ID: ChIJW6Rmul6TaDURleb5ghiERB4


In [67]:
# 청담동 대구뽈찜 아구찜, 좌표: (35.185, 129.083)
name = "청담동 대구뽈찜 아구찜"
lat = 35.185  # 예시 위도
lng = 129.083  # 예시 경도
place_id = get_place_id(api_key, name, lat, lng)
print("Place ID:", place_id)

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 35.1852158, 'lng': 129.0829483}, 'viewport': {'northeast': {'lat': 35.18656562989271, 'lng': 129.0842981298927}, 'southwest': {'lat': 35.18386597010727, 'lng': 129.0815984701073}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'icon_background_color': '#FF9E67', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet', 'name': '청담동대구뽈찜아구찜', 'opening_hours': {'open_now': False}, 'photos': [{'height': 4032, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/103579686833334083646">A Google User</a>'], 'photo_reference': 'AdCG2DMp3NdxTcnUB8Q-I2MATzTTHzRmIBIFRAPyf0zAVM_i85mTZHXu08Wg2mFVRu6fLwSbeOIRGWVIIPJ3YwJNdKiHS2MBT1mwafl_m_tNkMVSwEzQNWQEqrEpwjE6EjAUMfwB5A_zyQtdr4c42u5xb0Wfrrp6Jv6QdtzN0L98k3fZIRAT', 'width': 3024}], 'place_id': 'ChIJ75J6B0WTaDURinNqzY-fT7M', 'plus_code': {'compound_cod

In [54]:
name = "토곡정"
lat = 35.181  # 예시 위도
lng = 129.108  # 예시 경도

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "key": api_key,
    "location": f"{lat},{lng}",
    "radius": 100,  # 검색 반경 (미터 단위)
    "keyword": name,  # 레스토랑 이름
    "type": "restaurant"
}

response = requests.get(url, params=params)

In [64]:
response.json()['results'][2]

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 35.1805393, 'lng': 129.1073626},
  'viewport': {'northeast': {'lat': 35.18188912989272,
    'lng': 129.1087124298927},
   'southwest': {'lat': 35.17918947010727, 'lng': 129.1060127701073}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
 'icon_background_color': '#FF9E67',
 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet',
 'name': '김종구식맛치킨 토곡점',
 'opening_hours': {'open_now': False},
 'place_id': 'ChIJ0wciVeiTaDURFlxwWzbQu4c',
 'plus_code': {'compound_code': '54J4+6W Busan', 'global_code': '8Q7F54J4+6W'},
 'rating': 3.5,
 'reference': 'ChIJ0wciVeiTaDURFlxwWzbQu4c',
 'scope': 'GOOGLE',
 'types': ['restaurant', 'food', 'point_of_interest', 'establishment'],
 'user_ratings_total': 2,
 'vicinity': '102 Gwajeong-ro, Yeonje-gu'}

In [68]:
# 토곡정, 좌표: (35.181, 129.108)
name = "토곡정"
lat = 35.181  # 예시 위도
lng = 129.108  # 예시 경도

place_id = get_place_id(api_key, name, lat, lng)
print("Place ID:", place_id)

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 35.1840475, 'lng': 129.1087635}, 'viewport': {'northeast': {'lat': 35.18539732989273, 'lng': 129.1101133298927}, 'southwest': {'lat': 35.18269767010728, 'lng': 129.1074136701073}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'icon_background_color': '#FF9E67', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet', 'name': 'Obogjeong', 'opening_hours': {'open_now': False}, 'photos': [{'height': 3120, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116526748199148898816">박상배</a>'], 'photo_reference': 'AdCG2DMBX0ufbprx9CW-lVxmNRz7H5yNy6mLVuawWdpIlZv2UkdyVK3kPc_JDLxmIQM0j2Ul_M2mxYve1YaPYj9ObwG_zJSTi0aKYODVD2KtLK3qKKMD_PHqUw29nONjUwKfh51xbrFxXY18S3fL2wFWpVXNdz21N4UnG6RwmIB5C9oueIKz', 'width': 4160}], 'place_id': 'ChIJcTw0Ph-TaDUR7OHeS-fakjU', 'plus_code': {'compound_code': '54M5+J

In [ ]:
for i, row in none_id_df.iterrows():
    # time.sleep(0.1)
    restaurant_name = row["RESTAURANT_NAME_KOREAN"]
    location = f"{row['LATITUDE'],row['LONGITUDE']}" 
    print(f"식당이름: {restaurant_name}, 좌표: {location}")
    place_id = get_place_id(restaurant_name, location)
    if i > 20:
        break

In [69]:
get_place_details_new('ChIJ75J6B0WTaDURinNqzY-fT7M')

{'types': ['seafood_restaurant',
  'restaurant',
  'food',
  'point_of_interest',
  'establishment'],
 'formattedAddress': '12 World cup-daero 120beon-gil, Yeonje-gu, Busan, South Korea',
 'rating': 4.3,
 'regularOpeningHours': {'openNow': False,
  'periods': [{'open': {'day': 0, 'hour': 11, 'minute': 0},
    'close': {'day': 0, 'hour': 21, 'minute': 0}},
   {'open': {'day': 1, 'hour': 11, 'minute': 0},
    'close': {'day': 1, 'hour': 21, 'minute': 0}},
   {'open': {'day': 2, 'hour': 11, 'minute': 0},
    'close': {'day': 2, 'hour': 21, 'minute': 0}},
   {'open': {'day': 3, 'hour': 11, 'minute': 0},
    'close': {'day': 3, 'hour': 21, 'minute': 0}},
   {'open': {'day': 4, 'hour': 11, 'minute': 0},
    'close': {'day': 4, 'hour': 21, 'minute': 0}},
   {'open': {'day': 5, 'hour': 11, 'minute': 0},
    'close': {'day': 5, 'hour': 21, 'minute': 0}},
   {'open': {'day': 6, 'hour': 11, 'minute': 0},
    'close': {'day': 6, 'hour': 21, 'minute': 0}}],
  'weekdayDescriptions': ['Monday: 11:00\